# debug modeling

In [13]:
from pathlib import Path

import time
import shutil
import numpy as np
import pandas as pd
from Bio import SearchIO, SeqIO, SeqRecord, pairwise2
from Bio.Seq import Seq
from prody import parsePDB, writePDB

In [8]:
def correct_pdb_resnums(mol):
    """correct pdb residue number such as 1A,1B, and 1C

    Args:
        mol (prody.atomic.atomgroup.AtomGroup): pdb mol

    Returns:
        np.ndarray : correct residue number
    """
    new_resnum_list = []
    plus = 0
    resnum_before = float('inf')
    resid_before = float('inf')
    for resnum, resid in zip(mol.getResnums(), mol.getResindices()):
        if resnum_before == resnum and resid_before != resid:
            plus += 1
        new_resnum_list.append(resnum + plus)
        resnum_before = resnum
        resid_before = resid
    return np.array(new_resnum_list)

def remove_pdb_resnum_alphabet(pdb_file: str):
    """remove the alphabet from the residue number in pdb file

    Args:
        pdb_file (str) : pdb file path
    """
    with open(pdb_file, 'r') as f:
        lines = f.readlines()
        lines = list(filter(lambda line: len(line) > 0, lines))
        for i, line in enumerate(lines):
            if i > 0:
                line = line[:26] + ' ' + line[27:]
                lines[i] = line
    with open(pdb_file, 'w') as f:
        f.writelines(lines)

def download_template_pdb(pdb_id):
    template_pdb_dir = Path('template_pdb')
    template_pdb_file = (template_pdb_dir / pdb_id).with_suffix('.pdb')
    if not template_pdb_file.exists():
        pdb_file = str(Path(pdb_id.lower()).with_suffix('.pdb'))
        mol = parsePDB(pdb_id, compressed=False)
        correct_resnums = correct_pdb_resnums(mol)
        mol.setResnums(correct_resnums)
        mol = mol.select('not hetatm')
        writePDB(pdb_file, mol)
        remove_pdb_resnum_alphabet(pdb_file)
        shutil.move(pdb_file, template_pdb_file)
        time.sleep(3)
    return template_pdb_file

In [17]:
def convert_mol_to_seq(mol):
    resindices = mol.getResindices()
    resindices_diff = np.diff(resindices, prepend=float('inf'))
    res_start_indices = np.where(resindices_diff != 0)[0]
    seq = ''.join(np.array(list(mol.getSequence()))[res_start_indices])
    resnums = mol.getResnums()[res_start_indices]
    return seq, resnums

def convert_pdb_to_seq(pdb_path, chain):
    mol = parsePDB(pdb_path, chain=chain).select('not hetatm')
    seq, resnums = convert_mol_to_seq(mol)
    return seq, resnums

In [14]:
pdb_path = download_template_pdb('2ZCM')

In [23]:
convert_pdb_to_seq(pdb_path, 'A')

@> 1470 atoms and 1 coordinate set(s) were parsed in 0.01s.


('KDKIIDNAITLFSEKGYDGTTLDDISKSVNIKKASLYYHYDNKEEIYRKSVENCFNYFIDFYSIDGLYQFLFKFIFDVDERYIKLYVQLSSAPEALNSEIKHHLQEINTTLHDELIKYYDPTHIALDKEDFINILFLETWYFRASFSQKFGIIEDSKNRFKDQVYSLLNVFLK',
 array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
         41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
         54,  55,  56,  57,  58,  59,  60,  61,  62,  71,  72,  73,  74,
         75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 144, 145, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 1

In [18]:
mol = parsePDB('2ZCM')

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 2zcm downloaded (2zcm.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 3888 atoms and 1 coordinate set(s) were parsed in 0.02s.


In [19]:
convert_mol_to_seq(mol)

('MKDKIIDNAITLFSEKGYDGTTLDDISKSVNIKKASLYYHYDNKEEIYRKSVENCFNYFIDFMYSIDGLYQFLFKFIFDVDERYIKLYVQLSSAPEALNSEIKHHLQEINTTLHDELIKYYDPTHIALDKEDFINMILMFLETWYFRASFSQKFGIIEDSKNRFKDQVYSLLNVFLKMKDKIIDNAITLFSEKGYDGTTLDDISKSVNIKKASLYYHYDNKEEIYRKSVENCFNYFIDFMMRNYSIDGLYQFLFKFIFDVDERYIKLYVQLSSAPEALNSEIKHHLQEINTTLHDELIKYYDPTHIALDKEDFINMILMFLETWYFRASFSQKFGIIEDSKNRFKDQVYSLLNVFLKXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [22]:
def get_max_serial_position(np_array):
    start, end = 0, 0
    max_serial_len, tmp_len = 0, 0
    b_num = -2
    for i, num in enumerate(np_array):
        if num == b_num + 1:
            tmp_len += 1
        else:
            tmp_len = 1
            tmp_start = i
        b_num = num
        tmp_end = i
        if max_serial_len < tmp_len:
            start = tmp_start
            end = tmp_end
            max_serial_len = tmp_len
    return start, end

def align_seq(hseq, tseq):
    alignment_list = pairwise2.align.globalms(hseq, tseq, 5, -10, -2, -1)
    alignment = None
    var_indices_min = float('inf')
    for a in alignment_list:
        var_indices = np.var(np.where(np.array(list(a.seqA)) != '-')[0])
        if var_indices < var_indices_min:
            alignment = a
            var_indices_min = var_indices
    align_hseq, align_tseq = np.array(list(alignment.seqA)), np.array(list(alignment.seqB))
    align_hseq, align_tseq = align_hseq[np.where(align_tseq != '-')], align_tseq[np.where(align_hseq != '-')]
    align_hindices = np.where(align_tseq != '-')[0]
    align_tindices = np.where(align_hseq != '-')[0]
    t_start, t_end = get_max_serial_position(align_tindices)
    if t_end - t_start + 1 < len(hseq) * 0.8:
        raise(ValueError)
    t_not_serial = np.delete(np.arange(len(align_tindices)), np.arange(t_start, t_end + 1), 0)
    align_hindices = np.delete(align_hindices, t_not_serial, 0)
    align_tindices = np.delete(align_tindices, t_not_serial, 0)
    align_tseq[t_not_serial] = '-'
    return align_hseq, align_tseq, align_hindices, align_tindices

In [24]:
tseq = 'KDKIIDNAITLFSEKGYDGTTLDDISKSVNIKKASLYYHYDNKEEIYRKSVENCFNYFIDFYSIDGLYQFLFKFIFDVDERYIKLYVQLSSAPEALNSEIKHHLQEINTTLHDELIKYYDPTHIALDKEDFINILFLETWYFRASFSQKFGIIEDSKNRFKDQVYSLLNVFLK'
hseq = 'KDKIIDNAITLFSEKGYDGTTLDDISKSVNIKKASLYYHYDNKEEIYRKSVENCFNYFIDFXXRNHDDNYSIDG'

In [25]:
align_seq(hseq, tseq)

(array(['K', 'D', 'K', 'I', 'I', 'D', 'N', 'A', 'I', 'T', 'L', 'F', 'S',
        'E', 'K', 'G', 'Y', 'D', 'G', 'T', 'T', 'L', 'D', 'D', 'I', 'S',
        'K', 'S', 'V', 'N', 'I', 'K', 'K', 'A', 'S', 'L', 'Y', 'Y', 'H',
        'Y', 'D', 'N', 'K', 'E', 'E', 'I', 'Y', 'R', 'K', 'S', 'V', 'E',
        'N', 'C', 'F', 'N', 'Y', 'F', 'I', 'D', 'F', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', 'R', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', 'N', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-', '-', '-', '-', 'H', 'D', '-', '-', '-', '-',
        '-', '-', 'D', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', 'N', '-', '-', '-', '-', '-', '-', '-', 'Y', '-', '-',
        '-', 'S', '-', '-', '-', '-', '-', '-', 'I', '-', '-', 'D', '-',
        '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
        '-', '-', '-', '-'], dtype='<U1'),
 array([

In [26]:
pdb_path = download_template_pdb('1QDM')

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 1qdm downloaded (1qdm.pdb)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 9336 atoms and 1 coordinate set(s) were parsed in 0.06s.


In [27]:
mol = parsePDB(pdb_path)

@> 9336 atoms and 1 coordinate set(s) were parsed in 0.04s.


In [29]:
convert_mol_to_seq(mol)

('VRIALKKRPIDRNSRVATGLSGEEEGDIVALKNYMNAQYFGEIGVGTPPQKFTVIFDTGSSNLWVPSAKCYFSIACYLHSRYKAGASSTYKKNGKPAAIQYGTGSIAGYFSEDSVTVGDLVVKDQEFIEATKEPGITFLVAKFDGILGLGFKEISVGKAVPVWYKMIEQGLVSDPVFSFWLNRHVGGEIIFGGMDPKHYVGEHTYVPVTQKGYWQFDMGDVLVGGKSTGFCAGGCAAIADSGTSLLAGPTAIITEINEKIGAAGVVSQECKTIVSQYGQQILDLLLAETQPKKICSQVGLCTADPMCSACEMAVVWMQNQLAQNKTQDLILDYVNQLCNRLPSPMGESAVDCGSLGSMPDIEFTIGGKKFALKPEEYILKVGEGAAAQCISGFTAMDIPPPRGPLWILGDVFMGPYHTVFDYGKLRIGFAKAAVRIALKKRPIDRNSRVATGLSGEEEGDIVALKNYMNAQYFGEIGVGTPPQKFTVIFDTGSSNLWVPSAKCYFSIACYLHSRYKAGASSTYKKNGKPAAIQYGTGSIAGYFSEDSVTVGDLVVKDQEFIEATKEPGITFLVAKFDGILGLGFKEISVGKAVPVWYKMIEQGLVSDPVFSFWLNRHVGGEIIFGGMDPKHYVGEHTYVPVTQKGYWQFDMGDVLVGGKSTGFCAGGCAAIADSGTSLLAGPTAIITEINEKIGAAGVVSQECKTIVSQYGQQILDLLLAETQPKKICSQVGLCTADPMCSACEMAVVWMQNQLAQNKTQDLILDYVNQLCNRLPSPMGESAVDCGSLGSMPDIEFTIGGKKFALKPEEYILKVGEGAAAQCISGFTAMDIPPPRGPLWILGDVFMGPYHTVFDYGKLRIGFAKAAVRIALKKRPIDRNSRVATGLSGEEGDIVALKNYMNAQYFGEIGVGTPPQKFTVIFDTGSSNLWVPSAKCYFSIACYLHSRYKAGASSTYKKNGKPAAIQYGTGSIAGYFSEDSVTVGDLVVKDQEFIEATKE

In [30]:
convert_pdb_to_seq(pdb_path, 'A')

@> 3127 atoms and 1 coordinate set(s) were parsed in 0.02s.


('VRIALKKRPIDRNSRVATGLSGEEEGDIVALKNYMNAQYFGEIGVGTPPQKFTVIFDTGSSNLWVPSAKCYFSIACYLHSRYKAGASSTYKKNGKPAAIQYGTGSIAGYFSEDSVTVGDLVVKDQEFIEATKEPGITFLVAKFDGILGLGFKEISVGKAVPVWYKMIEQGLVSDPVFSFWLNRHVGGEIIFGGMDPKHYVGEHTYVPVTQKGYWQFDMGDVLVGGKSTGFCAGGCAAIADSGTSLLAGPTAIITEINEKIGAAGVVSQECKTIVSQYGQQILDLLLAETQPKKICSQVGLCTADPMCSACEMAVVWMQNQLAQNKTQDLILDYVNQLCNRLPSPMGESAVDCGSLGSMPDIEFTIGGKKFALKPEEYILKVGEGAAAQCISGFTAMDIPPPRGPLWILGDVFMGPYHTVFDYGKLRIGFAKAA',
 array([  6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
         19,  20,  21,  22,  23,  24,  25,  26,  27,   2,   3,   4,   5,
          6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
         19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
         45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
         58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79

In [31]:
download_template_pdb('1HVC')

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 1hvc downloaded (1hvc.pdb)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 3491 atoms and 1 coordinate set(s) were parsed in 0.03s.


PosixPath('template_pdb/1HVC.pdb')